In [6]:
import numpy as np
import os
import sys
sys.path.append('../')

from svm.data_process import load_data, loader_data
from anchor import anchor_tabular
from utils import get_default_config,ft_names
from svm.run_model import test,load_model

In [3]:
filename = '../data/FICO_final_data.csv'
train_loader,test_loader= loader_data(filename =filename)
save_path = '../data/svm_FICO_100.pth'
if(os.path.exists(save_path)):
    model = load_model(save_path)
else:
    print("Train Model First")

In [5]:
idx = 0
# data_set = train_loader.dataset.data['X'].cpu().numpy()
# x_sample, y_sample = train_loader.dataset[idx]
X_train,X_test,y_train,y_test = load_data(filename)
x_sample1 = X_train[idx]

# explainer = anchor_tabular.AnchorTabularExplainer([0,1], ft_names, data_set, categorical_names = {})
explainer1 = anchor_tabular.AnchorTabularExplainer([0,1], ft_names, X_train, categorical_names = {})
# explainer.fit(X_train.values, y_train, X_val.values, y_val, discretizer='quartile')

# decision = model.pred(x_sample)
# if decision < 0 :
#     decision = 0
# else:
#     decision = 1
# print('Prediction: ', explainer.class_names[decision])
# exp = explainer.explain_instance(x_sample.detach().numpy(), model.pred_numpy, threshold=0.95)
exp = explainer1.explain_instance(x_sample1, model.pred_numpy, threshold=0.95)

print('Anchor %d: %s' % (idx,' AND '.join(exp.names())))
print('Precision: %.2f' % exp.precision())
print('Coverage: %.2f' % exp.coverage())

Anchor 0: External Risk Estimate <= -0.82 AND % Trades Never Delq. <= -0.29
Precision: 0.98
Coverage: 0.15


In [9]:
# Get test examples where the anchora pplies
fit_anchor = np.where(np.all(X_test[:, exp.features()] == X_test[idx][exp.features()], axis=1))[0]
print('Anchor test precision: %.2f' % (np.mean(model.pred_numpy(X_test[fit_anchor]) == model.pred_numpy(X_test[idx].reshape(1, -1)))))
print('Anchor test coverage: %.2f' % (fit_anchor.shape[0] / float(X_test.shape[0])))

[ True False False ... False False False]
Anchor test precision: 1.00
Anchor test coverage: 0.00
